In [1]:
!pip3 install pytidylib tidy rdflib owlready2 Ontospy graphviz tqdm ipywidgets python-Levenshtein rdflib-sqlalchemy --user
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.nubank.com.br/pypi/, https://pypi.python.org/simple


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
from IPython.display import IFrame
from parserLattes3 import ParserLattes
import rdflib

from tqdm import tnrange, tqdm_notebook

In [39]:
import sys

ipython = get_ipython()


def hide_traceback(exc_tuple=None, filename=None, tb_offset=None, exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ''
#     return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))


ipython.showtraceback = hide_traceback

# Getting the data

In [4]:
basePath = "cache/" 
cvPath = "9788199690491510.html"
file = open(basePath+cvPath, 'r',  encoding="ISO-8859-1")
cvLattesHTML = file.read()

In [5]:
IFrame(basePath+cvPath, 800, 500)

In [6]:
import warnings

warnings.filterwarnings(action="once")

lattes_page = ParserLattes("9788199690491510", cvLattesHTML)

### Now, we need to create the triple store

# Schema.org

### WHat if we dont use the ontology from ontolattes and yes the springnature ontology???
 - Sources
   - http://www.linkeddatatools.com/semantic-modeling
   - https://scigraph.springernature.com/explorer/datasets/ontology/
   - https://medium.com/openlink-software-blog/sentences-notations-8b090cf28574#.8m00befqb
   - (THE MOST IMPORTANT LINK: https://scigraph.springernature.com/explorer/datasets/data_at_a_glance/)
     - Isso 'e estranho: Falando que nao usa mais ontologia e sim esse schema.org...

In [8]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF, RDF
import ontospy
import re

### Apparently Springernature is using the schema.org ontology to describe theire data now

In [12]:
schemaDotOrg = Namespace("http://schema.org/version/latest/schema.nt#")

In [13]:
schemaDotOrg.type

rdflib.term.URIRef('http://schema.org/version/latest/schema.nt#type')

## Queting Sparkql

In [18]:
qres = graph.query(
    """
    SELECT 
        ?author
        ?project
        ?year
        ?type
   WHERE {
      ?project schema:author ?uri .
      ?uri schema:givenName ?author .
      ?project schema:datePublished ?year .
      ?project schema:type ?type .
   }
""",
    initNs={"schema": schemaDotOrg},
)

for row in qres:
    print("%s published %s of type %s in %s" % row)

Diego published Por que é difícil fraudar o resultado eleitoral (O Povo). 2018. (Programa de rádio ou of type 2018 in http://schema.org/version/latest/schema.nt#TechArticle
Diego published Voto impresso, o retorno ao passado que opõe Bolsonaro e a Procuradoria Geral (El País). 2018. (Programa de rádio ou of type 2018 in http://schema.org/version/latest/schema.nt#TechArticle
Diego published Declaração de Bolsonaro sobre fraude em urnas é 'desinformação', diz procurador (Huffpost Brasil). 2018. (Programa de rádio ou of type 2018 in http://schema.org/version/latest/schema.nt#TechArticle
Diego published Brazilian government reiterates e-voting security (ZDNet). 2018. (Programa de rádio ou of type 2018 in http://schema.org/version/latest/schema.nt#TechArticle
Diego published Reportagem mostra como funciona app de mensagens alvo da Justiça (Fantástico). 2016. (Programa de rádio ou of type 2016 in http://schema.org/version/latest/schema.nt#TechArticle
Diego published Site publica informações 

In [ ]:
graph.close()  # https://rdflib.readthedocs.io/en/stable/persistence.html

# Making a function for adding triples from a given lattes page

In [ ]:
lattes_page = ParserLattes("9788199690491510", cvLattesHTML)

In [28]:
def add_lattes_reasercher_to_graph(graph, lattes_page, schema):
    import hashlib

    from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
    from rdflib.namespace import DC, FOAF, RDF
    from toolz import curry

    @curry
    def add_non_duplicated(triple_or_quad, graph):
        ### Maybe using a try catch
        if triple_or_quad not in graph:
            graph.add(triple_or_quad)

    add_triple = add_non_duplicated(graph=graph)

    uri = lattes_page.idMembro
    name = lattes_page.nomeCompleto
    first_name = name.split(" ")[0]
    family_name = " ".join(name.split(" ")[1:])
    gender = lattes_page.sexo

    researcher = URIRef(uri)

    add_triple((researcher, schema.type, schema.Person))
    add_triple((researcher, schema.name, Literal(name)))
    add_triple((researcher, schema.familyName, Literal(family_name)))
    add_triple((researcher, schema.givenName, Literal(first_name)))
    add_triple((researcher, schema.gender, Literal(gender)))

    # Check if the ID in listaIDLattesColaboradores is the same as uri (as in uri = lattes_page.idMembro)
    for colaborador in sorted(lattes_page.listaIDLattesColaboradores):
        # I assume the colaborator will be added to type Person in a different iteration.
        add_triple((researcher, schema.knows, URIRef(colaborador)))

    for idioma in lattes_page.listaIdioma:
        nome_idioma = idioma.nome
        add_triple((URIRef(nome_idioma), schema.type, schema.Language))
        add_triple((researcher, schema.knowsLanguage, Literal(nome_idioma)))

    for formacao_academica in lattes_page.listaFormacaoAcademica:
        tipo_formacao = formacao_academica.tipo
        instituicao = formacao_academica.nomeInstituicao
        instituicao_md5 = hashlib.md5(instituicao.encode("utf-8")).hexdigest()

        add_triple((URIRef(instituicao_md5), schema.type, schema.EducationalOrganization))
        add_triple((URIRef(instituicao_md5), schema.name, Literal(instituicao)))
        add_triple((researcher, schema.hasCredential, Literal(tipo_formacao)))
        add_triple((researcher, schema.alumniOf, Literal(instituicao)))

    for artigo_periodico in lattes_page.listaArtigoEmPeriodico:
        doi_url = artigo_periodico.doi
        # Added springer nature format (we can remove later if needed)
        doi = re.sub("http://dx.doi.org/", "", doi_url)
        sameAsSpriger = "http://scigraph.springernature.com/" + "pub." + re.sub("http://dx.doi.org/", "", doi_url)
        # We are not using 'autores' for now.
        autores = artigo_periodico.autores
        ano = artigo_periodico.ano
        titulo = artigo_periodico.titulo
        genre = "artigo em periodico"

        if doi != "":
            add_triple((URIRef(doi), schema.name, Literal(titulo)))
            add_triple((URIRef(doi), schema.type, schema.ScholarlyArticle))
            add_triple((URIRef(doi), schema.author, researcher))
            add_triple((URIRef(doi), schema.datePublished, Literal(ano)))
            add_triple((URIRef(doi), schema.genre, Literal(genre)))
            add_triple((URIRef(doi), schema.sameAs, URIRef(sameAsSpriger)))

    for artigo_aceito in lattes_page.listaArtigoAceito:
        doi_url = artigo_periodico.doi
        doi = re.sub("http://dx.doi.org/", "", doi_url)
        # Added springer nature format (we can remove later if needed)
        sameAsSpriger = "http://scigraph.springernature.com/" + "pub." + re.sub("http://dx.doi.org/", "", doi_url)
        autores = artigo_aceito.autores
        ano = artigo_aceito.ano
        titulo = artigo_aceito.titulo
        genre = "artigo aceito para publicacao"

        if doi != "":
            add_triple((URIRef(doi), schema.type, schema.ScholarlyArticle))
            add_triple((URIRef(doi), schema.author, researcher))
            add_triple((URIRef(doi), schema.datePublished, Literal(ano)))
            add_triple((URIRef(doi), schema.name, Literal(titulo)))
            add_triple((URIRef(doi), schema.genre, Literal(genre)))
            add_triple((URIRef(doi), schema.sameAs, URIRef(sameAsSpriger)))

    for projeto_de_pesquisa in lattes_page.listaProjetoDePesquisa:
        ano = projeto_de_pesquisa.ano
        descricao = projeto_de_pesquisa.descricao
        nome = projeto_de_pesquisa.nome
        nome_md5 = hashlib.md5(nome.encode("utf-8")).hexdigest()

        add_triple((URIRef(nome_md5), schema.name, Literal(nome)))
        add_triple((URIRef(nome_md5), schema.type, schema.ResearchProject))
        # Maybe this property does not exist
        add_triple((URIRef(nome_md5), schema.year, Literal(ano)))
        add_triple((URIRef(nome_md5), schema.description, Literal(descricao)))
        add_triple((URIRef(nome_md5), schema.author, researcher))

    for trabalho_tecnico in lattes_page.listaTrabalhoTecnico:
        # We are not using 'autores' for now.
        autores = trabalho_tecnico.autores
        ano = trabalho_tecnico.ano
        titulo = trabalho_tecnico.titulo
        genre = "trabalho tecnico"
        nome_md5 = hashlib.md5(titulo.encode("utf-8")).hexdigest()

        add_triple((URIRef(nome_md5), schema.name, Literal(titulo)))
        add_triple((URIRef(nome_md5), schema.type, schema.TechArticle))
        add_triple((URIRef(nome_md5), schema.datePublished, Literal(ano)))
        add_triple((URIRef(nome_md5), schema.author, researcher))
        add_triple((URIRef(nome_md5), schema.genre, Literal(genre)))

    for livro in lattes_page.listaLivroPublicado:
        ano = livro.ano
        titulo = livro.titulo
        # We are not using 'autores' for now.
        autores = livro.autores
        edicao = livro.edicao
        editora = livro.editora
        paginas = livro.paginas
        nome_md5 = hashlib.md5(titulo.encode("utf-8")).hexdigest()

        add_triple((URIRef(nome_md5), schema.name, Literal(titulo)))
        add_triple((URIRef(nome_md5), schema.type, schema.Book))
        add_triple((URIRef(nome_md5), schema.datePublished, Literal(ano)))
        # Use researcher or 'autores'?
        add_triple((URIRef(nome_md5), schema.author, researcher))
        add_triple((URIRef(nome_md5), schema.bookEdition, Literal(edicao)))
        add_triple((URIRef(nome_md5), schema.editor, Literal(editora)))
        add_triple((URIRef(nome_md5), schema.numberOfPages, Literal(paginas)))

    for capitulo_livro in lattes_page.listaCapituloDeLivroPublicado:
        ano = capitulo_livro.ano
        titulo = capitulo_livro.titulo
        # We are not using 'autores' for now.
        autores = capitulo_livro.autores
        editora = capitulo_livro.editora
        paginas = capitulo_livro.paginas
        livro = capitulo_livro.livro
        nome_md5 = hashlib.md5(titulo.encode("utf-8")).hexdigest()

        add_triple((URIRef(nome_md5), schema.name, Literal(titulo)))
        add_triple((URIRef(nome_md5), schema.type, schema.Book))
        add_triple((URIRef(nome_md5), schema.datePublished, Literal(ano)))
        # Use researcher or 'autores'?
        add_triple((URIRef(nome_md5), schema.author, researcher))
        add_triple((URIRef(nome_md5), schema.isPartOf, Literal(livro)))
        add_triple((URIRef(nome_md5), schema.editor, Literal(editora)))
        add_triple((URIRef(nome_md5), schema.pagination, Literal(paginas)))

In [29]:
from rdflib import plugin
from rdflib.store import Store
from rdflib_sqlalchemy import registerplugins
import os

registerplugins()

schema = Namespace("http://schema.org/version/latest/schema.nt#")

SQLALCHEMY_URL = "sqlite:///%(here)s/development.sqlite" % {"here": os.getcwd()}
print(f"Creating the file to output: {SQLALCHEMY_URL}")
store = plugin.get("SQLAlchemy", Store)()

graph = Graph(store)
graph.open(SQLALCHEMY_URL, create=True)

Creating the file to output: sqlite:////root/work/experiments/tcc/python_lattes_parser/pythonLattes/development.sqlite


1

In [30]:
add_lattes_reasercher_to_graph(graph, lattes_page, schema)

In [31]:
qres = graph.query(
    """
    SELECT 
        ?author
        ?name
        ?year
        ?type
    WHERE {
        ?project schema:author ?uri .
        ?uri schema:givenName ?author .
        ?project schema:datePublished ?year .
        ?project schema:type ?type .
        ?project schema:name ?name .
    }
""",
    initNs={"schema": schema},
)

for row in qres:
    print("%s published %s in %s of type %s" % row)

Diego published Building Secure Protocols for Extensible Distributed Coordination through Secure Extensions in 2019 of type http://schema.org/version/latest/schema.nt#ScholarlyArticle
Diego published The Return of Software Vulnerabilities in the Brazilian Voting Machine in 2019 of type http://schema.org/version/latest/schema.nt#ScholarlyArticle
Diego published Efficient and secure software implementations of Fantomas in 2019 of type http://schema.org/version/latest/schema.nt#ScholarlyArticle
Diego published A framework for searching encrypted databases in 2018 of type http://schema.org/version/latest/schema.nt#ScholarlyArticle
Diego published Fog orchestration for the Internet of Everything: state-of-the-art and research challenges in 2018 of type http://schema.org/version/latest/schema.nt#ScholarlyArticle
Diego published CRPUF: A modeling-resistant delay PUF based on cylindrical reconvergence in 2018 of type http://schema.org/version/latest/schema.nt#ScholarlyArticle
Diego published R

Diego published Por qué el voto electrónico falló en Brasil (El Fueguino). 2018. (Programa de rádio ou in 2018 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Urna eletrônica: As ações da Justiça Eleitoral contra fraudes no sistema (HuffPost Brasil). 2018. (Programa de rádio ou in 2018 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Los riesgos del voto electrónico (Notícias SER). 2018. (Programa de rádio ou in 2018 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Você confia na urna eletrônica? (Segunda Chamada). 2018. (Programa de rádio ou in 2018 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Reforma electoral en la ciudad (NCN). 2018. (Programa de rádio ou in 2018 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Os cientistas explicam como as urnas eletrônicas podem ser mais seguras (ICMC). 2018. (Programa de rádio ou in 2018 of t

Diego published Urnas eletrônicas brasileiras são testadas e apresentam novas falhas (TecMundo). 2017. (Programa de rádio ou in 2017 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Nova quebra de segurança conseguiu alterar software da urna eletrônica (Convergência Digital). 2017. (Programa de rádio ou in 2017 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Reportagem mostra como funciona app de mensagens alvo da Justiça (Fantástico). 2016. (Programa de rádio ou in 2016 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Vulnerabilidades, Criptografia e Urnas Eletrônicas - Parte I (SecurityCast). 2016. (Programa de rádio ou in 2016 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Vulnerabilidades, Criptografia e Urnas Eletrônicas - Parte II (SecurityCast). 2016. (Programa de rádio ou in 2016 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published

Diego published Aplicativo para celular vai fiscalizar a urna eletrônica (Jornal de Londrina). 2014. (Programa de rádio ou in 2014 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Você Fiscal: aplicativo quer apuração paralela das eleições (Contas Abertas). 2014. (Programa de rádio ou in 2014 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Campanha vai criar aplicativo para fiscalizar eleições (Correio Braziliense). 2014. (Programa de rádio ou in 2014 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Professores da Unicamp e do MIT desenvolvem aplicativos para eleições mais transparentes (O Globo). 2014. (Programa de rádio ou in 2014 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego published Brasileiros colocam a mão no bolso para fiscalizar eleição e cobrar eleitos (IG). 2014. (Programa de rádio ou in 2014 of type http://schema.org/version/latest/schema.nt#TechArticle
Diego pu

## Parse all the cache folder and load it to graph

In [32]:
from os import listdir
from os.path import isfile, join

In [33]:
basePath = "./cache/"

files_names = [f for f in listdir(basePath) if isfile(join(basePath, f))]

In [34]:
not_lattes_pages = []
for lattes_page in files_names:
    with open(basePath + lattes_page, encoding="iso-8859-1") as f:
        if "possivel baixar" in f.read():
            not_lattes_pages.append(lattes_page)

print(len(not_lattes_pages))

7788


In [35]:
lattes_profile_list = [x for x in files_names if x not in not_lattes_pages]

In [37]:
schema = Namespace("http://schema.org/version/latest/schema.nt#")

SQLALCHEMY_URL = "sqlite:///%(here)s/knowlattes_graph_final.sqlite" % {"here": os.getcwd()}
print(f"Creating the file to output: {SQLALCHEMY_URL}")
store = plugin.get("SQLAlchemy", Store)()

new_graph = Graph(store)
new_graph.open(SQLALCHEMY_URL, create=True)

Creating the file to output: sqlite:////root/work/experiments/tcc/python_lattes_parser/pythonLattes/knowlattes_graph_final.sqlite


1

In [ ]:
%%time
number_profiles_iterartion = 10000 # len(lattes_profile_list) #100 # If loading al the files, pass len(lattes_profile_list)

for lattes_page in tqdm_notebook(lattes_profile_list[:number_profiles_iterartion]):
    lattes_id = re.sub('.html', '', lattes_page)
    
    file = open(basePath+lattes_page, 'r',  encoding="ISO-8859-1")
    lattes_file = file.read()
    file.close()

    lattes_page = ParserLattes(lattes_id, lattes_file)
    
    add_lattes_reasercher_to_graph(new_graph, lattes_page, schema)

In [41]:
print("Done")

Done


In [42]:
new_graph.serialize(destination='./new_graph.n3', format='n3')

In [43]:
len(new_graph)

/root/.local/lib/python3.6/site-packages/rdflib_sqlalchemy/sql.py:56: SADeprecationWarning: The FromClause.count() method is deprecated, and will be removed in a future release.   Please use the functions.count function available from the func namespace.
  selectClause = table.count(whereClause)


45207

In [25]:
new_graph.close()

## Loading the Graph

In [ ]:
%%time

schema = Namespace("http://schema.org/version/latest/schema.nt#")

SQLALCHEMY_URL = "sqlite:///%(here)s/knowlattes_graph.sqlite" % {"here": os.getcwd()}
print(f"{SQLALCHEMY_URL}")
store = plugin.get("SQLAlchemy", Store)()

graph = Graph(store)
graph.open(SQLALCHEMY_URL, create=True)

In [ ]:
for i in graph:
    print(i)

In [ ]:
qres = graph.query("""
    SELECT 
        ?author
        ?name
        ?year
        ?type
    WHERE {
        ?project schema:author ?uri .
        ?uri schema:givenName ?author .
        ?project schema:datePublished ?year .
        ?project schema:type ?type .
        ?project schema:name ?name .
    }
""",
    initNs={'schema': schema}
)

### Merging graphs
 - https://rdflib.readthedocs.io/en/stable/merging.html

# Testing